In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# Setting up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.page_load_strategy = "eager"

In [3]:
# To Automatically download and use the correct ChromeDriver
service = Service(ChromeDriverManager().install())

# To Initialize WebDriver
driver = webdriver.Chrome(service=service, options=options)

In [4]:
# Home URL
BASE_URL = "https://www.boston.gov/events?page="

In [5]:
event_list = []

In [6]:
# To Loop through the first 3 pages
for page in range(3):
    if page % 2 == 0:
        driver.quit()
        driver = webdriver.Chrome(service=service, options=options)

    url = BASE_URL + str(page)
    print(f"Scraping page: {page + 1} -> {url}")
    driver.get(url)

    try:
        WebDriverWait(driver, 60).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        print(f"Page {page + 1} loaded successfully!")
    except Exception as e:
        print(f"Error on page {page + 1}: {e}")
        continue

    # Parsing the extracted content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.find_all("article", class_="calendar-listing-wrapper")

    # All the different events are under article component
    for article in articles:
        title_tag = article.find("div", class_="title")
        title = title_tag.text.strip() if title_tag else "No Title"

        details_link_tag = article.find("a", class_="button")
        details_link = "https://www.boston.gov" + details_link_tag["href"] if details_link_tag else "No Link"

        event_list.append({"Title": title, "Event Details Link": details_link})

    time.sleep(3)

# Close the browser
driver.quit()

Scraping page: 1 -> https://www.boston.gov/events?page=0
Page 1 loaded successfully!
Scraping page: 2 -> https://www.boston.gov/events?page=1
Page 2 loaded successfully!
Scraping page: 3 -> https://www.boston.gov/events?page=2
Page 3 loaded successfully!


In [7]:
# Converting list to DataFrame
df_events = pd.DataFrame(event_list)

In [8]:
# To scrape through individual event page to extract specific informations

In [9]:
import re

In [10]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [11]:
event_details_list = []

In [12]:
import requests

In [13]:
# Loop to get individual pages - Event details
for index, row in df_events.iterrows():
    URL = row["Event Details Link"]

    # Checking if the URL follows a particular format - other URLs are skipped
    if not re.match(r"^https://www\.boston\.gov/node/\d+$", URL):
        print(f"Skipping invalid URL: {URL}")
        continue

    print(f"Scraping: {URL}")

    response = requests.get(URL, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to fetch page: {response.status_code}")
        continue 

    soup = BeautifulSoup(response.text, "lxml")

    # To Extract Title
    title_tag = soup.find("h1")
    title = title_tag.text.strip() if title_tag else "No Title"

    # To Extract Start Date
    start_date_tag = soup.find("div", class_="date-title t--intro")
    start_date = start_date_tag.text.strip() if start_date_tag else "No Start Date"

    # To Extract Event Timings (Full Date & Time Info)
    timing_tag = soup.find("div", class_="detail-item__content")
    event_timing = timing_tag.get_text(strip=True, separator=" ") if timing_tag else "No Event Timings"

    # To Extract Location
    location_tag = soup.find("div", class_="addr-a street-block")
    location = location_tag.text.strip() if location_tag else "No Location"

    # To Extract Contact Person
    contact_tag = soup.find("div", class_="detail-item__body detail-item__body--secondary field-item")
    contact = contact_tag.text.strip() if contact_tag else "No Contact Info"

    # To Extract Email
    email_tag = soup.find("a", href=lambda x: x and x.startswith("mailto:"))
    email = email_tag["href"].replace("mailto:", "") if email_tag else "No Email"

    # To Extract Admission Price**
    admission_price = "No Price Info"
    
    # To Find all 'li' elements with class 'dl-i evt-sb-i' which contain the relevant labels
    details_list = soup.find_all("li", class_="dl-i evt-sb-i")

    for detail in details_list:
        label_tag = detail.find("div", class_="dl-t")  # Finds the label like "Price:"
        value_tag = detail.find("div", class_="detail-item__body--secondary field-item", role="listitem")  # The actual value
        
        if label_tag and value_tag:
            label_text = label_tag.get_text(strip=True).lower()
            value_text = value_tag.get_text(strip=True)

            if "price" in label_text:
                admission_price = value_text  # ✅ Extracts only Admission Price

    # To Extract Event Type
    event_type_tag = soup.find("a", href=re.compile(r"/event-type/"))
    event_type = event_type_tag.text.strip() if event_type_tag else "No Event Type"

    # To Extract Description
    description_tag = soup.find("div", class_="body")
    description = description_tag.get_text(strip=True, separator=" ") if description_tag else "No Description"

    # To Store event details in a dictionary
    event_data = {
        "Title": title,
        "Start Date": start_date,  
        "Event Timings": event_timing, 
        "Location": location,
        "Contact": contact,
        "Email": email,
        "Admission Price": admission_price,  # ✅ Corrected extraction
        "Event Type": event_type,
        "Description": description,
        "Event URL": URL
    }

    # To Append event details to list
    event_details_list.append(event_data)

    time.sleep(1)

Scraping: https://www.boston.gov/node/16062456
Scraping: https://www.boston.gov/node/16274781
Scraping: https://www.boston.gov/node/16045031
Scraping: https://www.boston.gov/node/16274741
Scraping: https://www.boston.gov/node/16172861
Scraping: https://www.boston.gov/node/16062491
Scraping: https://www.boston.gov/node/16273371
Scraping: https://www.boston.gov/node/16161706
Skipping invalid URL: https://www.boston.govhttps://www.zoomgov.com/meeting/register/L7Sp_72sR8eriih76EtEfQ#/registration
Scraping: https://www.boston.gov/node/16350556
Scraping: https://www.boston.gov/node/16102411
Scraping: https://www.boston.gov/node/16273371
Scraping: https://www.boston.gov/node/16105806
Scraping: https://www.boston.gov/node/16350706
Scraping: https://www.boston.gov/node/16350566
Scraping: https://www.boston.gov/node/16274716
Scraping: https://www.boston.gov/node/16274961
Scraping: https://www.boston.gov/node/16045001
Scraping: https://www.boston.gov/node/16292961
Scraping: https://www.boston.gov

In [14]:
# To Convert list to DataFrame
df_events_details = pd.DataFrame(event_details_list)

In [15]:
df_events_details

,Title,Start Date,Event Timings,Location,Contact,Email,Admission Price,Event Type,Description,Event URL
0,Parks Fitness: Virtual Zumba class with Lena,"January 11, 2025","10:00am - 11:00am Repeats weekly on Saturday, ...",VIRTUAL CLASS,Jennifer Misiaszek,mayor24reply@boston.gov,No Price Info,Sports and health,Perfect For Everybody and every body! Each Zum...,https://www.boston.gov/node/16062456
1,Jamaica Pond Nature Center,"February 15, 2025",11:00am - 2:00pm,507 Jamaicaway\n ...,Park Rangers,mayor24reply@boston.gov,No Price Info,Art,Join the Boston Park Rangers at the Jamaica Po...,https://www.boston.gov/node/16274781
2,Parks Fitness: Virtual HIIT class with Jacqui,"January 5, 2025","8:00am - 9:00am Repeats weekly on Sunday, star...",VIRTUAL CLASS,Jennifer Misiaszek,mayor24reply@boston.gov,No Price Info,Sports and health,This class is a total body workout that will g...,https://www.boston.gov/node/16045031
3,Snowshoeing Safari,"February 16, 2025",1:00pm - 2:00pm,1 Circuit Dr.\n ...,Park Rangers,mayor24reply@boston.gov,No Price Info,Art,Join the Boston Park Rangers for a snowshoeing...,https://www.boston.gov/node/16274741
4,Age Strong Chair Yoga,"February 17, 2025","9:00am - 10:00am Repeats weekly on Monday, sta...",Online,Nuria Silva,mayor24reply@boston.gov,No Price Info,Sports and health,Join the online event For more information on ...,https://www.boston.gov/node/16172861
...,...,...,...,...,...,...,...,...,...,...
72,Age Strong Latin Dance,"February 18, 2025","11:30am - 12:30pm Repeats weekly on Tuesday, s...",Online,Nuria Silva,mayor24reply@boston.gov,No Price Info,Sports and health,Join the online Event For more information on ...,https://www.boston.gov/node/16105806
73,City Hall On The Go: Charlestown,"February 25, 2025",12:30pm - 2:00pm,Charlestown Branch of the Boston Public Librar...,Office Civic Organizing,mayor24reply@boston.gov,No Price Info,Civic Engagement,Our vibrant City Hall On The Go truck has a sc...,https://www.boston.gov/node/16350641
74,Jamaica Pond Nature Center,"February 25, 2025",1:00pm - 3:00pm,507 Jamaicaway\n ...,Park Rangers,mayor24reply@boston.gov,No Price Info,Art,Join the Boston Park Rangers at the Jamaica Po...,https://www.boston.gov/node/16274796
75,Parks Fitness: Virtual Yoga class with Nikita,"January 7, 2025","5:30pm - 6:30pm Repeats weekly on Tuesday, sta...",VIRTUAL CLASS,Jennifer Misiaszek,mayor24reply@boston.gov,No Price Info,Sports and health,"This is a joyful, energizing Vinyasa practice ...",https://www.boston.gov/node/16045001


### Now  let's load the data into Snowflake

In [17]:
SNOWFLAKE_ACCOUNT=''
SNOWFLAKE_USER=''
SNOWFLAKE_PASSWORD=''
SNOWFLAKE_DATABASE=''
SNOWFLAKE_SCHEMA=''
SNOWFLAKE_WAREHOUSE=''
SNOWFLAKE_ROLE=''

In [18]:
import snowflake.connector


conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT, 
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA,
    role=SNOWFLAKE_ROLE,
    client_session_keep_alive=True
)


cursor = conn.cursor()

In [19]:
df_events_details.columns

Index(['Title', 'Start Date', 'Event Timings', 'Location', 'Contact', 'Email',
       'Admission Price', 'Event Type', 'Description', 'Event URL'],
      dtype='object')

In [20]:
# Rename DataFrame columns to match Snowflake table
df_events_details = df_events_details.rename(columns={
    "Start Date": "Start_Date",
    "Event Timings": "Event_Timings",
    "Admission Price": "Admission_Price",
    "Event Type": "Event_Type",
    "Event URL": "Event_URL",
    "Title": "Event_Title"
})


In [21]:
insert_query = """
INSERT INTO BOSTON_GOV_EVENT_DETAILS 
(Event_Title, Start_Date, Event_Timings, Location, Contact, Email, Admission_Price, Event_Type, Description, Event_URL)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


In [22]:
data_to_insert = [tuple(row) for row in df_events_details.itertuples(index=False, name=None)]

In [23]:
cursor.executemany(insert_query, data_to_insert)

conn.commit()
print(f"Inserted {len(data_to_insert)} records into Snowflake.")

cursor.close()
conn.close()


Inserted 77 records into Snowflake.
